# 💬🌽 Semantic Kernel Agents 

## 📦 Get the right packages together

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.1-alpha"

Installed Packages Microsoft.SemanticKernel, 1.18.1-rc Microsoft.SemanticKernel.Agents.Core, 1.18.1-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.18.1-alpha Microsoft.SemanticKernel.Plugins.Core, 1.18.1-alpha

In [5]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.Extensions.Logging;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;
using Microsoft.Extensions.DependencyInjection;
using System.Threading;

// Load settings
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

## 🔌 Create a Semantic Kernel plugin

In [6]:
public class TimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

## 🤖 Create a chat completion agent

In [7]:
#pragma warning disable SKEXP0001

// Create the kernel
IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(
    "gpt-4o",
    apiKey);
// builder.Plugins.AddFromType<TimePlugin>();
Microsoft.SemanticKernel.Kernel kernel = builder.Build();

#pragma warning disable SKEXP0110
// Create the agent
ChatCompletionAgent agent =
    new()
    {
        Name = "TimeAgent",
        Instructions = "You can tell the time to the user",
        Kernel = kernel,
    };

// Start the chat
ChatHistory chat = [];
while (true)
{
    // Get the user input
    string input = await InteractiveKernel.GetInputAsync("Your message");
    if (input == "bye")
    {
        break;
    }
    chat.Add(new ChatMessageContent(AuthorRole.User, input));
    Console.WriteLine("User > " + input);

    // Get the assistant response
    await foreach (ChatMessageContent message in agent.InvokeAsync(chat))
    {
        chat.Add(message);
        Console.WriteLine("Assistant > " + message);
    }
}

User > Hello
Assistant > Hello! How can I assist you today?


## 🤖 Create an assistant agent

In [9]:
// Create the kernel
IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
//builder.Plugins.AddFromType<TimePlugin>();
Microsoft.SemanticKernel.Kernel assistantKernel = builder.Build();

#pragma warning disable SKEXP0110
// Create the agent
OpenAIAssistantAgent agent =
    await OpenAIAssistantAgent.CreateAsync(
        kernel: assistantKernel,
        OpenAIClientProvider.ForOpenAI(apiKey),
        new("gpt-4-turbo")
        {
            Name = "<agent name>",
            Instructions = "<agent instructions>",
            EnableCodeInterpreter = true,
        });

// Start the chat
string threadId = await agent.CreateThreadAsync();
while (true)
{
    // Get the user input
    string input = await InteractiveKernel.GetInputAsync("Your message");
    if (input == "bye")
    {
        break;
    }
    await agent.AddChatMessageAsync(threadId, new ChatMessageContent(AuthorRole.User, input));
    Console.WriteLine("User > " + input);

    // Get the assistant response
    await foreach (ChatMessageContent message in agent.InvokeAsync(threadId))
    {
        Console.WriteLine("Assistant > " + message);
    }
}

User > Hello
Assistant > Hello! How can I assist you today?


## 💬 Using assistants in a group chat

In [11]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

// Define the agents
ChatCompletionAgent agentReviewer =
    new()
    {
        Instructions = """
            You are an art director who has opinions about copywriting born of a love for David Ogilvy.
            The goal is to determine if the given copy is acceptable to print.
            If so, state that it is approved.
            If not, provide insight on how to refine suggested copy without example.
            """,
        Name = "ArtDirector",
        Kernel = kernel,
    };

ChatCompletionAgent agentWriter =
    new()
    {
        Instructions = """
            You are a copywriter with ten years of experience and are known for brevity and a dry humor.
            The goal is to refine and decide on the single best copy as an expert in the field.
            Only provide a single proposal per response.
            You're laser focused on the goal at hand.
            Don't waste time with chit chat.
            Consider suggestions when refining an idea.
            """,
        Name = "CopyWriter",
        Kernel = kernel,
    };

// Define a termination strategy for the chat
class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}

// Create a chat for agent interaction
AgentGroupChat chat =
    new(agentWriter, agentReviewer)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [agentReviewer],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    }
            }
    };

// Invoke chat and display messages
string input = "concept: a way for multiple AI agents to collaborate on a single task";
chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (ChatMessageContent content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

Console.WriteLine($"# IS COMPLETE: {chat.IsComplete}");

# user: 'concept: a way for multiple AI agents to collaborate on a single task'
# Assistant - CopyWriter: '"Many minds, one mission."'
# Assistant - ArtDirector: 'Approved. This copy is succinct and conveys the collaborative essence effectively.'
# IS COMPLETE: True
